In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import seaborn

df = pd.read_csv("../datasets/energy_dataset.csv")
print(df.columns)
#print(df.head)
columns = ["generation hydro pumped storage aggregated", "forecast wind offshore eday ahead", "generation fossil coal-derived gas", "generation wind offshore", "generation marine", "generation geothermal",
"generation fossil peat","generation fossil oil shale","forecast solar day ahead","forecast wind onshore day ahead"]
df = df.drop(columns,axis = 1)
#plt.matshow(df.corr())
df.fillna(df.interpolate(method="linear"),inplace=True)
#plt.show()
#df.mean()
#df.var()
df["time"] = pd.to_datetime(df["time"],format = "%Y-%m-%d %H:%M:%S")
df['time'] = df['time'].apply(lambda x: x.replace(tzinfo=None))
df["time"] = pd.to_datetime(df["time"],format="ISO8601")
df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month
df["week"] = df["time"].dt.weekday
df = df.set_index('time')
print(df["week"].value_counts(0)) 


In [ ]:
#rename columns 

columns = []
for name in df.columns:

    if "generation" in name:
        renamed =  name.replace("generation","")
        df = df.rename(columns = {name: renamed})

df.columns

In [ ]:
"""splitting data"""
print(df.columns)
x = df.drop("total load actual",axis = 1)
y = df["total load actual"]
tss = TimeSeriesSplit(n_splits=2)
print(tss)
for train,test in tss.split(df):
    trainx, testx = x.iloc[train,:], x.iloc[test,:]
    trainy, testy = y.iloc[train], y.iloc[test]

len(testx)






In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
def boxplots(time_period):
    fig,axs = plt.subplots(figsize=(10,8))
    data = zip(df[time_period], df["total load actual"])
    seaborn.boxplot(data = df,x = time_period,y = "total load actual", ax = axs)
    axs.tick_params(axis='x', colors='white')
    axs.tick_params(axis='y', colors='white')
    axs.set_title("total load actual",color = "white")

    plt.show()

boxplots("month")

In [ ]:
def feature_plot(feature):
    plt.figure(figsize=(8, 6))
    plt.tick_params(axis='x', colors='white')
    plt.tick_params(axis='y', colors='white')
    plt.plot(df.index, df[feature])

def multi_plot(features):
    fig,axs = plt.subplots(len(features),figsize=(16,48))
    for x in range(len(features)):
        axs[x].plot(df.index, df[features[x]])
        axs[x].set_title(features[x],color = "white")
        axs[x].tick_params(axis='x', colors='white')
        axs[x].tick_params(axis='y', colors='white')
        
    fig.tight_layout()

feature_plot("total load actual")
#multi_plot(["generation biomass","generation fossil brown coal/lignite","generation fossil gas"])



In [ ]:
#plot all features
fig = plt.figure(figsize=(30,18))
ax = fig.add_subplot(2, 1, 1)
for column in df:
    ax.plot(df.index, df[column])
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.set_yscale("log")
ax.legend()